In [1]:
MODEL_NAME = 'intfloat/multilingual-e5-small'

WORKING_DIR = '/home/hotchpotch/src/huggingface.co/datasets/hotchpotch/wikipedia-passages-jawiki-embeddings/'

WIKIPEDIA_DS = 'singletongue/wikipedia-utils'
WIKIPEDIA_DS_NAME = 'passages-c400-jawiki-20230403'
# DS_NAME = 'hotchpotch/wikipedia-ja-20231030'
INDEX_NAME = 'faiss_indexes/passages-c400-jawiki-20230403/multilingual-e5-small-passage/index_m96_mbit8_nlist512.faiss'


In [2]:
from datasets.download import DownloadManager
from datasets import load_dataset

ds = load_dataset(path=WIKIPEDIA_DS, name=WIKIPEDIA_DS_NAME, split="train")
dm = DownloadManager()
# index_pass  = dm.download(f"https://huggingface.co/datasets/{DS_NAME}/resolve/main/{INDEX_NAME}")


/home/yu1/miniconda3/envs/llm-sc/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
ds

Dataset({
    features: ['id', 'pageid', 'revid', 'title', 'section', 'text'],
    num_rows: 5555583
})

In [5]:
from sentence_transformers import SentenceTransformer

MODEL = SentenceTransformer(MODEL_NAME, device='cuda')
MODEL.max_seq_length = 512

In [6]:
import faiss
index = faiss.read_index(WORKING_DIR + INDEX_NAME)
index.ntotal

5555583

In [25]:
index.nprobe = 128

In [26]:
import numpy as np
prefix = ""

if '-e5-' in INDEX_NAME:
    prefix = "query: "

def to_emb(text, prefix=prefix) -> np.ndarray:
    return MODEL.encode([prefix + text], normalize_embeddings=True) # type: ignore
 
emb = to_emb('フリーレンの声優は誰?')
emb.shape


(1, 384)

In [51]:
import time
def faiss_search_by_text(text, ds, faiss_index = index, top_k=10):
    emb = to_emb(text)
    start_time = time.time()
    D, I = faiss_index.search(emb, top_k)
    end_time = time.time()
    print(f"Search took {end_time - start_time:.2f} seconds")
    results = []
    for idx, (id, d) in enumerate(zip(I[0], D[0])):
        if idx >= 0:
            id = int(id)
            results.append((ds[id], d))
    return results

# q = 'フリーレンの声優は誰？'
# q = 'フリーレンの悪役は?'
q = '宝川温泉が東の横綱、湯原温泉が西の横綱と呼ばれる、温泉地ではおなじみの野外に設置されたお風呂を何というでしょう?'
# q = '日本で最大の湖は?'

for data, score in faiss_search_by_text(q, ds):
    print(score, data['title'], data['text'])
    # print(list(data.keys()))



Search took 0.04 seconds
0.18711293 露天風呂 露天風呂番付(ろてんぶろばんづけ)は、温泉番付の一種であり、露天風呂が有名な温泉地を番付したものである。作成されたのは1981年で、野口冬人の手で編集された。露天風呂番付における西の横綱は湯原温泉の「砂湯」、東の横綱は宝川温泉である。湯原温泉には大きな露天風呂番付のモニュメントが掲示されており、温泉地のシンボルとなっている。
0.22631446 水上温泉郷 昭和の温泉評論家・野口冬人が1980年(昭和55年)発行の旅行誌上で発表した、国内温泉地における「露天風呂番付」では、東の横綱に選ばれている。旅行口コミサイト「トリップアドバイザー」の選ぶ「外国人が最も注目した日本の観光スポット2008」においては、草津温泉、伊香保温泉等を押さえて第16位に選ばれた。2013年にロイター通信社が選ぶ「世界の温泉トップ10」の第6位に選出(日本の温泉として唯一の選出)される と、ロイターの影響度の高さも相まって外国人観光客の間で知名度が飛躍的に高まった。2014年(平成26年)4月26日公開の東宝映画『テルマエ・ロマエII』では主要なロケ地の一つになった (cf.) 。
0.22801468 水上温泉郷 詳しくは、主人公であるローマ帝国のテルマエ設計技師ルシウス(演:阿部寛)がタイムスリップして出没する“平たい顔族(日本人もしくは大和民族のこと)”の露天風呂が、汪泉閣で最も大きい露天風呂「子宝の湯」である。宝川(たからがわ)は、利根川上流域の支流の一つであり、一級河川である。群馬県利根郡みなかみ町藤原と新潟県南魚沼市藤原との県境に位置する中央分水界を成す大烏帽子山の、南麓を源流域として発し、おおよそ南東方向へ流れて、宝台樹山(水上宝台樹スキー場を擁する山)の北西麓に広がる扇状地にある奥利根橋付近 で利根川本流 に合流する。上述のとおり、「宝川温泉」の名の由来になっている。宝泉峡(ほうせんきょう)は、宝川の下流域にある渓谷。宝川温泉から800mほど上流にある。総延長約2km。
0.22832566 湯原温泉 現在、利用されている温泉はその一部だけでほとんどは未利用のまま川に流されている。河原に遊歩道及び駐車場が整備されている。基本は湯治、保養、療養向けであるが、場所柄、山陰・山陽の行楽拠点としても重宝